## Hyperparameter Tuning

Select base values for hyperparameter automatically. We want to modify the parameters which make up this model and see how they affect the accuracy.

In [ ]:
dropout_rate = 0.2
regularization_rate = 0.01

def model_tune(hparams):
  model = tf.keras.Sequential([InputLayer(input_shape=(IM_SIZE, IM_SIZE, 3)),

                              Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu',
                                      kernel_regularizer=L2(hparams[HP_REGULARIZATION_RATE])),
                              BatchNormalization(),
                              MaxPool2D(pool_size=2, strides=2),
                              Dropout(rate=hparams[HP_DROPOUT]),

                              Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu',
                                      kernel_regularizer=L2(hparams[HP_REGULARIZATION_RATE])),
                              BatchNormalization(),
                              MaxPool2D(pool_size=2, strides=2),

                              Flatten(),

                              Dense(hparams[HP_NUM_UNITS_1], activation='sigmoid', kernel_regularizer=L2(hparams[HP_REGULARIZATION_RATE])), #
                              BatchNormalization(),
                              Dropout(rate=hparams[HP_DROPOUT]),

                              Dense(hparams[HP_NUM_UNITS_2], activation='sigmoid', kernel_regularizer=L2(hparams[HP_REGULARIZATION_RATE])),
                              BatchNormalization(),
                              Dense(1, activation='sigmoid')]) # 0 or 1
  model.compile(optimizer=Adam(learning_rate=hparams[HP_LEARNING_RATE]),
              loss='binary_crossentropy',
              metrics=['accuracy'])
  model.fit(val_dataset, epochs=1, verbose=1)
  _, accuracy = model.evaluate(val_dataset)

  return accuracy


In [ ]:
HP_NUM_UNITS_1 = hp.HParam('num_units_1', hp.Discrete([16,32,64,128]))
HP_NUM_UNITS_2 = hp.HParam('num_units_2', hp.Discrete([16,32,64,128]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1,0.2,0.3]))
HP_REGULARIZATION_RATE = hp.HParam('regularization_rate', hp.Discrete([0.001,0.01,0.1]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-4,1e-3]))

### Grid Search

- fix range of value
- if range is large, can take so long

In [ ]:
run_number = 0
for num_units_1 in HP_NUM_UNITS_1.domain.values:
  for num_units_2 in HP_NUM_UNITS_2.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
      for regularization_rate in HP_REGULARIZATION_RATE.domain.values:
        for learning_rate in HP_LEARNING_RATE.domain.values:

          hparams = {
              HP_NUM_UNITS_1: num_units_1,
              HP_NUM_UNITS_2: num_units_2,
              HP_DROPOUT: dropout_rate,
              HP_REGULARIZATION_RATE: regularization_rate,
              HP_LEARNING_RATE: learning_rate
          }

          file_writer = tf.summary.create_file_writer(f'logs/{run_number}')
          with file_writer.as_default():
            hp.hparams(hparams)
            accuracy = model_tune(hparams)
            tf.summary.scalar('accuracy', accuracy, step=0)
          print(f"Run {run_number} : Current hparams is: ", hparams)

          run_number += 1

173/173 [==============================] - 15s 45ms/step - loss: 0.7503 - accuracy: 0.5132
Run 0 : Current hparams is:  {HParam(name='num_units_1', domain=Discrete([16, 32, 64, 128]), display_name=None, description=None): 16, HParam(name='num_units_2', domain=Discrete([16, 32, 64, 128]), display_name=None, description=None): 16, HParam(name='dropout', domain=Discrete([0.1, 0.2, 0.3]), display_name=None, description=None): 0.1, HParam(name='regularization_rate', domain=Discrete([0.001, 0.01, 0.1]), display_name=None, description=None): 0.001, HParam(name='learning_rate', domain=Discrete([0.0001, 0.001]), display_name=None, description=None): 0.0001}
173/173 [==============================] - 15s 45ms/step - loss: 0.8697 - accuracy: 0.4988
Run 1 : Current hparams is:  {HParam(name='num_units_1', domain=Discrete([16, 32, 64, 128]), display_name=None, description=None): 16, HParam(name='num_units_2', domain=Discrete([16, 32, 64, 128]), display_name=None, description=None): 16, HParam(name=

In [ ]:
tensorboard --logdir="./logs/"